<img src='filmes_natal.jpg'>

# Problema

Obter uma lista de filmes sobre o tema Natal no site iMDb e, para cada filme, as seguintes informações:

* Título
* link iMDb com mais informações sobre o filme
* Sinópse
* Diretor
* Gênero
* País
* Idioma
* Tempo de duração em minutos
* Atores principais
* Score
* Ano

Antes de atacarmos esse problema, precisamos entender algumas coisas básicas.

# Componentes de uma página web

Quando visitamos uma página, nosso navegador faz uma solicitação à um servidor de Internet. Essa solicitação é chamada de solicitação GET, pois estamos recebendo arquivos do servidor. O servidor então envia arquivos que dizem ao navegador como renderizar a página para nós. Os arquivos se enquadram em alguns tipos principais:

* HTML: Contém o conteúdo principal da página;
* CSS: Adiciona estilos para que a página fique mais bonita;
* JS: Arquivos JavaScript adicionam interatividade à página;
* Imagens: formatos de imagem, tais como JPG e PNG permitem que páginas web exibam imagens.

Após nosso navegador receber todos os arquivos, ele renderiza a página e a exibe para nós. Existem muitas coisas que acontecem no background para renderizar bem uma página, mas não precisamos nos preocupar com a maioria delas quando estamos realizando o web scraping. Quando realizamos o web scraping, estamos interessados no conteúdo principal da página, então olharemos para o HTML.

# Entendendo HTML

HyperText Markup Language (HTML) é uma linguagem com que as páginas web são criadas. O HTML não é uma linguagem de programação, como o Python – em vez disso, é uma linguagem de marcação que diz ao navegador como exibir o conteúdo. O HTML permite que você faça coisas similares ao Microsoft Word – criar parágrafos, deixar textos em negrito etc. Por não ser uma linguagem de programação, o HTML nem se aproxima da complexidade do Python.

O HTML é formado por elementos chamados tags. Veja uma exemplo de código em HTML:

```HTML
<html>
    <head>
    </head>
    <body>
        <p>
            Aqui um exemplo de parágrafo com texto.
            <a href="https://www.dataquest.io">Learn Data Science Online</a>
        </p>
        <p>
            Aqui um segundo exemplo de parágrafo com texto.
            <a href="https://www.python.org">Python</a>
        </p>
    </body>
</html>
```

No exemplo acima, temos duas tags a. As tags a são links, e dizem ao navegador para renderizar uma outra página. A propriedade href determina para onde o link vai. As tags a e p são extremamente comuns, a seguir temos outras:

* div – indica uma divisão, uma área na página;
* b – deixa qualquer texto dentro dela em negrito;
* i – deixa em itálico;
* table – cria uma tabela;
* form – cria um formulário;

Para uma lista completa de tags, clique [aqui](https://developer.mozilla.org/pt-BR/docs/Web/HTML/HTML5/HTML5_element_list).

# A biblioteca requests

A primeira coisa que precisamos fazer para realizar o web scrap é baixar a página. Podemos baixar as páginas utilizando a biblioteca requests do Python. A biblioteca requests fará uma solicitação GET ao servidor, que fará o download dos conteúdos HTML da página solicitada para nós. Existem vários tipos de solicitação diferentes que podemos realizar utilizando a biblioteca requests – GET é apenas um deles.

# A Biblioteca BeautifulSoup

Depois de baixamos um documento HTML da página que desejamos usando o requests. Nós podemos utilizar a biblioteca BeautifulSoup para analisar esse documento e extrair o conteúdo das tags que desejamos.

# Importando as Bibliotecas

In [3]:
import requests # para se conectar a página web
from bs4 import BeautifulSoup # para extrair informações do html
import pandas as pd # para manusear os dados e salva-los em csv 

# Obtendo os títulos e links dos Filmes

In [4]:
# url que lista filmes sobre o tema natal 
url_fonte = 'http://www.imdb.com/find?q=Christmas&s=tt&ref_=fn_tt'

In [13]:
# obtendo conteúdo da página
con = requests.get(url_fonte)

In [14]:
# criando objeto BeautifulSoup 
soup = BeautifulSoup(con.content, "html.parser")

In [7]:
titulos_filmes = [div.find('a').contents[0] for div in soup.findAll('td', attrs={'class':'result_text'})]

In [8]:
links_filmes = ['http://www.imdb.com' + div.a['href'] for div in soup.findAll('td', attrs={'class':'result_text'})]

In [21]:
links_filmes

[u'http://www.imdb.com/title/tt1410207/?ref_=fn_tt_tt_1',
 u'http://www.imdb.com/title/tt0217978/?ref_=fn_tt_tt_2',
 u'http://www.imdb.com/title/tt0144827/?ref_=fn_tt_tt_3',
 u'http://www.imdb.com/title/tt0097958/?ref_=fn_tt_tt_4',
 u'http://www.imdb.com/title/tt0085334/?ref_=fn_tt_tt_5',
 u'http://www.imdb.com/title/tt0388419/?ref_=fn_tt_tt_6',
 u'http://www.imdb.com/title/tt1083448/?ref_=fn_tt_tt_7',
 u'http://www.imdb.com/title/tt6273004/?ref_=fn_tt_tt_8',
 u'http://www.imdb.com/title/tt6359956/?ref_=fn_tt_tt_9',
 u'http://www.imdb.com/title/tt7608418/?ref_=fn_tt_tt_10',
 u'http://www.imdb.com/title/tt7030416/?ref_=fn_tt_tt_11',
 u'http://www.imdb.com/title/tt5902138/?ref_=fn_tt_tt_12',
 u'http://www.imdb.com/title/tt0369436/?ref_=fn_tt_tt_13',
 u'http://www.imdb.com/title/tt6225520/?ref_=fn_tt_tt_14',
 u'http://www.imdb.com/title/tt0047673/?ref_=fn_tt_tt_15',
 u'http://www.imdb.com/title/tt7038614/?ref_=fn_tt_tt_16',
 u'http://www.imdb.com/title/tt1067106/?ref_=fn_tt_tt_17',
 u'htt

# Obtendo as informações de cada filme

### Sinópse

In [147]:
url_filme = 'http://www.imdb.com/title/tt1410207/?ref_=fn_tt_tt_1'

In [148]:
# obtendo conteúdo da página
con = requests.get(url_filme)

# criando objeto BeautifulSoup 
soup = BeautifulSoup(con.content, "html.parser")

In [16]:
# <div class="summary_text" itemprop="description">
sinopse =  soup.find('div', attrs={'class':'summary_text'})

In [19]:
print sinopse.text        


                    The Griswold family's plans for a big family Christmas predictably turn into a big disaster.
            


In [31]:
# Fazendo função que irá capturar essa informação para nós

def get_sinopse(soup):
    '''
    Função rebe um objeto BeautifulSoup
    chamado soup com o html da página do filme
    desejado e retorna uma string da sinópse do filme.
    Caso não haja uma sinópse, retorna None.
    '''
    sinopse =  soup.find('div', attrs={'class':'summary_text'})
    
    if sinopse.text:
        return sinopse.text
    else:
        return None

In [32]:
# testando função

url_filme_com_sinopse = 'http://www.imdb.com/title/tt0097958/?ref_=fn_tt_tt_4'
url_filme_sem_sinopse = 'http://www.imdb.com/title/tt1410207/?ref_=fn_tt_tt_1'

urls = [url_filme_com_sinopse, url_filme_sem_sinopse]

for url in urls:
    con = requests.get(url)
    soup = BeautifulSoup(con.content, "html.parser")
    print get_sinopse(soup)


                    The Griswold family's plans for a big family Christmas predictably turn into a big disaster.
            

Add a Plot »
            


### Diretor

In [36]:
# <span class="itemprop" itemprop="name">Jeremiah S. Chechik</span>
diretor =  soup.find('span', attrs={'class':'itemprop'}, itemprop='name')

In [38]:
print diretor.text

Sebastián Lelio


In [39]:
def get_diretor(soup):
    '''
    retorna uma lista com 
    os gêneros do filme
    '''
    diretor =  soup.find('span', attrs={'class':'itemprop'}, itemprop='name')
    if diretor.text:
        return diretor.text
    else:
        return None

### Gênero

In [62]:
# tags com as informações do Gênero dos filmes
'''
<div class="see-more inline canwrap" itemprop="genre">
            <h4 class="inline">Genres:</h4>
<a href="/genre/Comedy?ref_=tt_stry_gnr"> Comedy</a>
        </div>
'''

div =  soup.find('div', attrs={'class':'see-more inline canwrap'}, itemprop='genre')

In [59]:
generos = [a.text for a in div.find_all('a')]

In [60]:
generos

[u' Drama', u' Romance']

In [61]:
def get_generos(soup):
    div =  soup.find('div', attrs={'class':'see-more inline canwrap'}, itemprop='genre')
    if div.a:
        div =  soup.find('div', attrs={'class':'see-more inline canwrap'}, itemprop='genre')
        generos = [a.text for a in div.find_all('a')]
    else:
        return None

### País

In [115]:
'''
    <div class="txt-block">
    <h4 class="inline">Country:</h4>
        <a href="/search/title?country_of_origin=cl&amp;ref_=tt_dt_dt" itemprop="url">Chile</a>
              <span class="ghost">|</span>
        <a href="/search/title?country_of_origin=fr&amp;ref_=tt_dt_dt" itemprop="url">France</a>
'''


tags_paises = soup.select("a[href*=country_of_origin]")

In [121]:
tags_paises

[<a href="/search/title?country_of_origin=cl&amp;ref_=tt_dt_dt" itemprop="url">Chile</a>,
 <a href="/search/title?country_of_origin=fr&amp;ref_=tt_dt_dt" itemprop="url">France</a>]

In [122]:
len(tags_paises)

2

In [117]:
paises = [a.text for a in tags_paises]

In [118]:
paises

[u'Chile', u'France']

In [123]:
def get_paises(soup):
    tags_paises = soup.select("a[href*=country_of_origin]")
    if len(tags_paises):
        paises = [a.text for a in tags_paises]
        return paises
    else:
        return None

### Idioma

In [124]:
'''
<a href="/search/title?title_type=feature&amp;primary_language=es&amp;sort=moviemeter,
asc&amp;ref_=tt_dt_dt" itemprop="url">Spanish</a>
'''

tags_idioma = soup.select("a[href*=primary_language]")

In [128]:
tags_idioma

[<a href="/search/title?title_type=feature&amp;primary_language=es&amp;sort=moviemeter,asc&amp;ref_=tt_dt_dt" itemprop="url">Spanish</a>]

In [129]:
tags_idioma[0].text

u'Spanish'

In [130]:
def get_idioma(soup):
    tags_idioma = soup.select("a[href*=primary_language]")
    if tags_idioma[0].text:
        return tags_idioma[0].text
    else:
        None

### Tempo de duração em minutos

In [134]:
'''
<time itemprop="duration" datetime="PT99M">99 min</time>
'''

tempo = soup.findAll('time', itemprop="duration")

In [136]:
tempo

[<time datetime="PT99M" itemprop="duration">\n                        1h 39min\n                    </time>,
 <time datetime="PT99M" itemprop="duration">99 min</time>,
 <time datetime="PT104M" itemprop="duration">104 min</time>]

In [138]:
t = tempo[1].text

In [140]:
t.find(' min')

2

In [142]:
t2 = t[:t.find(' min')]
t2

u'99'

In [143]:
int(t2)

99

In [144]:
def get_duracao(soup):
    tempo = soup.findAll('time', itemprop="duration")
    t = tempo[1].text
    return int(t[:t.find(' min')])

### Atores Principais

In [166]:
'''
<span itemprop="actors" itemscope="" itemtype="http://schema.org/Person">
<a href="/name/nm1443757?ref_=tt_ov_st_sm" itemprop="url"><span class="itemprop" itemprop="name">Manuela Martelli</span></a>,             </span>
'''
atores = soup.findAll('span', itemprop="actors")

In [167]:
atores

[<span itemprop="actors" itemscope="" itemtype="http://schema.org/Person">\n<a href="/name/nm1443757?ref_=tt_ov_st_sm" itemprop="url"><span class="itemprop" itemprop="name">Manuela Martelli</span></a>,             </span>,
 <span itemprop="actors" itemscope="" itemtype="http://schema.org/Person">\n<a href="/name/nm2926981?ref_=tt_ov_st_sm" itemprop="url"><span class="itemprop" itemprop="name">Diego Ruiz</span></a>,             </span>,
 <span itemprop="actors" itemscope="" itemtype="http://schema.org/Person">\n<a href="/name/nm3396723?ref_=tt_ov_st_sm" itemprop="url"><span class="itemprop" itemprop="name">Alicia Rodr\xedguez</span></a> </span>]

In [168]:
for div in atores:
    print div.a.text

Manuela Martelli
Diego Ruiz
Alicia Rodríguez


In [169]:
def gets_atores(soup):
    divs_atores = soup.findAll('span', itemprop="actors")
    atores = [div.a.text for div in divs_atores]
    return atores

### Score

In [170]:
'''
<span itemprop="ratingValue">6,2</span>
'''

score = soup.find('span', itemprop="ratingValue")

In [173]:
float(score.text)

6.2

In [174]:
def get_score(soup):
    score = soup.find('span', itemprop="ratingValue")
    return float(score.text)

### Ano

In [177]:
# <a href="/year/2009/?ref_=tt_ov_inf">2009</a>

ano = soup.select("a[href*=/year/]")

In [180]:
ano

[<a href="/year/2009/?ref_=tt_ov_inf">2009</a>]

In [181]:
ano[0].text

u'2009'

In [182]:
def get_ano(soup):
    ano = soup.select("a[href*=/year/]")
    return int(ano[0].text)

# Obtendo os Dados para Todos os Filmes